In [ ]:
#This code learn ranking based on fairness def of CTR, and evaluate based on fairness def of CTR with trust bias (Compass)

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
import numpy as np
import pickle as pkl
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from YahooDataReader import YahooDataReader
import math
import statistics
import itertools
from random import seed, shuffle
import os,sys
import urllib.request  as urllib2
from collections import defaultdict
from sklearn import preprocessing
from IPython.display import display
from collections import Counter

In [ ]:
#Compas

SEED = 1234
seed(SEED)
np.random.seed(SEED)

def add_intercept(x):

    """ Add intercept to the data before linear classification """
    m,n = x.shape
    intercept = np.ones(m).reshape(m, 1) # the constant b
    return np.concatenate((intercept, x), axis = 1)

In [ ]:
def load_compas_data():

    FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"] #features to be used for classification
    CONT_VARIABLES = ["priors_count"] # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "two_year_recid" # the decision variable
    SENSITIVE_ATTRS = ["race"]


    COMPAS_INPUT_FILE = "../data/compas.csv"
    #check_data_file(COMPAS_INPUT_FILE)

    # load the data and get some stats
    df = pd.read_csv(COMPAS_INPUT_FILE)
    df
    df = df.dropna(subset=["days_b_screening_arrest"]) # dropping missing vals

    # convert to np array
    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])


    """ Filtering the data """

    # These filters are the same as propublica (refer to https://github.com/propublica/compas-analysis)
    # If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense. 
    idx = np.logical_and(data["days_b_screening_arrest"]<=30, data["days_b_screening_arrest"]>=-30)


    # We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
    idx = np.logical_and(idx, data["is_recid"] != -1)

    # In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
    idx = np.logical_and(idx, data["c_charge_degree"] != "O") # F: felony, M: misconduct

    # We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
    idx = np.logical_and(idx, data["score_text"] != "NA")

    # we will only consider blacks and whites for this analysis
    idx = np.logical_and(idx, np.logical_or(data["race"] == "African-American", data["race"] == "Caucasian"))

    # select the examples that satisfy this criteria
    for k in data.keys():
        data[k] = data[k][idx]

    #display(data)

    """ Feature normalization and one hot encoding """

    # convert class label 0 to -1
    y = data[CLASS_FEATURE]
    #y[y==0] = -1



    print ("\nNumber of people recidivating within two years")
    print (pd.Series(y).value_counts())
    print ("\n")


    X = np.array([]).reshape(len(y), 0) # empty array with num rows same as num examples, will hstack the features to it
    x_control = defaultdict(list)

    feature_names = []
    for attr in FEATURES_CLASSIFICATION:
        print ("attr", attr)
        vals = data[attr]
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            vals = preprocessing.scale(vals) # 0 mean and 1 variance  
            vals = np.reshape(vals, (len(y), -1)) # convert from 1-d arr to a 2-d arr with one col

        else: # for binary categorical variables, the label binarizer uses just one var instead of two
            lb = preprocessing.LabelBinarizer()
            lb.fit(vals)
            vals = lb.transform(vals)
    
        # add to sensitive features dict
        if attr in SENSITIVE_ATTRS:
            print ("True")
            x_control[attr] = vals


        # add to learnable features
        X = np.hstack((X, vals))

        if attr in CONT_VARIABLES: # continuous feature, just append the name
            feature_names.append(attr)
        else: # categorical features
            if vals.shape[1] == 1: # binary features that passed through lib binarizer
                feature_names.append(attr + "_" + str(lb.classes_[1]))
            else:
                for k in lb.classes_: # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))

    print ("feature_names", feature_names)
    # convert the sensitive feature to 1-d array
    x_control = dict(x_control)
    for k in x_control.keys():
        assert(x_control[k].shape[1] == 1) # make sure that the sensitive feature is binary after one hot encoding
        x_control[k] = np.array(x_control[k]).flatten()
    print ("x_control", x_control)
    # sys.exit(1)

    #"""permute the date randomly"""
    #perm = list(range(0,X.shape[0])
    #shuffle(perm)
    #X = X[perm]
    #y = y[perm]
    #for k in x_control.keys():
    #x_control[k] = x_control[k][perm]


    X = add_intercept(X)

    feature_names = ["intercept"] + feature_names
    assert(len(feature_names) == X.shape[1])
    print ("Features we will be using for classification are:", feature_names, "\n")

    # wrtie data in csv file

    #A = pd.DataFrame(list(x_control.items()),columns=['index','race'])
    #A.to_csv('compas_A.csv', sep='\t') 
    y = pd.DataFrame(data=y,columns=['two_year_recid'])
    y.to_csv('compas_Y.csv', sep='\t') 
    X1 = pd.DataFrame(data=X[0:,0:],columns=feature_names) #X[0,1:] #index=X[0:,0]
    #print ("X1B", X1.keys())
    A = X1.race_Caucasian
    #print ("A", A)
    X1.drop(['race_Caucasian'], axis=1) #in place-- does not work.So X1 is unchanged
    print ("X1", X1.keys())
    #X = pd.DataFrame(list(X.items()),columns=feature_names)
    X1.to_csv('compas_X.csv', sep='\t')
    A.to_csv('compas_A.csv', sep='\t')
    data = pd.concat([A,X1],axis = 1).values #race_caucasian column is added to ll data. So this feature appears twice, once at index=0 and also at index=5. Here, we consider cocuasians to be bottom rank in criminal search
    #print ("data", data)
    return data, y


In [ ]:
def make_sample_record(orig_data, orig_label):

    #data, to_protect, label = load_compas_data() #read_arrhythmia(shuffle=True, dropmissing=True)

    data = orig_data
    orig_label = np.array(orig_label)
    label = orig_label

    data = pd.DataFrame(data)
    label = pd.DataFrame(label)
    display(data)


    Y = np.array(label)
    Y = np.squeeze(Y)

    #data = pd.concat([to_protect,data],axis = 1).values

    X = np.array(data)

    num_feats = X.shape[1]
    numX = X.shape[0]


    datasize = 500
    cs_size = 25
    split_on_doc = 0.8
    testsize = 100
    ratio_relevant = 0.4
    ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)

    # generate a candidate set of size 10 everytime
    data_X = []
    data_Y = []
    test_X = []
    test_Y = []
    group_identities_train = []
    group_identities_test = []
    print("Sampling between 0 and {} for train".format(numX*split_on_doc))
    p = ratios_col[0:int(numX*split_on_doc)]
    p = p / sum(p)
    for i in range(datasize):
        cs_indices = np.random.choice(np.arange(0, int(numX*split_on_doc), dtype=int), size=cs_size, p=p)
        cs_X = X[cs_indices]
        cs_Y = Y[cs_indices]
        data_X.append(cs_X)
        data_Y.append(cs_Y)
        group_identities_train.append(cs_X[:,0]) #index=0 shows caucasian= whites
    print("Sampling between {} and {} for test".format(int(numX*split_on_doc), numX))
    p = ratios_col[int(numX*split_on_doc):]
    p = p/sum(p)
    docID_test= []
    while len(docID_test) < testsize: #used while instead of for to make sure given "continue", we still have len(testsize) qid#
        cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
        print ("cs_indices",cs_indices)
        #*START: added to skip the sets that has zero female or male or all female rel score 0 or all male relscore 0*#
        group_id= X[cs_indices,0]
        print ("group_id", group_id)
        if (len(set(group_id))==1 or 
            sum([Y[item] for i, item in enumerate(cs_indices) if group_id[i]==0]) == 0 or
            sum([Y[item] for i, item in enumerate(cs_indices) if group_id[i]==1]) == 0):
            #print ("continue")
            continue
        #*END: added to skip the sets that has zero female or male*#
        docID_test.append (cs_indices.tolist())
        print ("docID_test", docID_test)
        test_X.append(X[cs_indices])
        test_Y.append(Y[cs_indices])
        group_identities_test.append(X[cs_indices,0])
    

    return data_X, data_Y, test_X, test_Y, docID_test

In [ ]:
# Write data

data, label = load_compas_data()
#permuted_indices = np.random.permutation(len(data)) #permuted_data
data_X, data_Y, test_X, test_Y, docID_test= make_sample_record(data, label)
pkl.dump((data_X, data_Y), open("compas_train_rank_25.pkl", "wb"))
pkl.dump((test_X, test_Y), open("compas_test_rank_25.pkl", "wb"))
pickle.dump(docID_test, open( "compas_docID_test_25.txt", "wb" ) )


In [ ]:
data_df = pd.DataFrame(data)
MaleID= data_df[data_df[0]==1].index
FemaleID= data_df[data_df[0]==0].index
MaleID_str=[]
FemaleID_str=[]
for i in MaleID:
    MaleID_str.append(str(i))
for i in FemaleID:
    FemaleID_str.append(str(i))

In [ ]:
#Read data

docID_test = pickle.load( open( "compas_docID_test_25.txt", "rb" ) )

with open('compas_train_rank_25.pkl', 'rb') as f:
    data_train = pickle.load(f)

with open('compas_test_rank_25.pkl', 'rb') as f:
     data_test = pickle.load(f)

In [ ]:
#Learn rel-scores

dr = YahooDataReader(None)
dr.data = pkl.load(open("compas_train_rank_25.pkl", "rb"))
vdr = YahooDataReader(None)
vdr.data = pkl.load(open("compas_test_rank_25.pkl","rb"))                  
model = linear_model.LinearRegression(fit_intercept=False, normalize=False)
feats, rel = dr.data
feats = np.array([item for sublist in feats for item in sublist])
rel = np.array([item for sublist in rel for item in sublist])
model.fit(feats, rel)
# predictions on validation
feats, rel = vdr.data
se_sum = 0
length = 0
predicted_rels = []
for i, query in enumerate(feats):
    rel_pred = model.predict(query)

            
    predicted_rels.append(rel_pred)
    se_sum += np.sum((rel_pred - rel[i])**2)
    length += len(rel[i])
#print (type(predicted_rels)) #predicted_rels has 100 subarray, each with 25 rel scores#

In [ ]:
#This code learn ranking based on fairness def of CTR, and evaluate based on fairness def of CTR with trust bias (German Credit)

pd.set_option('display.max_rows', None)
df = pd.read_csv("../data/german_credit_data.csv", index_col=0)
df = df.fillna(value="NA")
df

In [ ]:
MaleID= df.query('Sex == "male"').index
FemaleID= df.query('Sex == "female"').index
MaleID_str=[]
FemaleID_str=[]

for i in MaleID:
    MaleID_str.append(str(i))
for i in FemaleID:
    FemaleID_str.append(str(i))


In [ ]:
#Write data

preprocess = make_column_transformer((StandardScaler(),['Age', 'Credit amount', 'Duration']),
    (OneHotEncoder(sparse=False), ['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk']))
print ("type",type(df))
mat = preprocess.fit_transform(df)
mat[996, :]   #row 10 all columns (11th row in df)#


X = mat[:, :-2]   #all rows of mat, without last two columns. X is an array containg 1000 subarray each with 29 elements#
Y = mat[:, -1]  #all rows of mat, only last column. Y is an array contating 1000 element#
num_feats = X.shape[1]
# print (num_feats)
numX = X.shape[0]
# print (numX)

datasize = 500 #number of qid in train
cs_size = 25 #number of docs under each qid
split_on_doc = 0.8
testsize = 100 #number of qid in test
ratio_relevant = 0.4
ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)
# print (ratios_col)

# generate a candidate set of size 25 everytime for each qid
data_X = []
data_Y = []
test_X = []
test_Y = []
group_identities_train = []
group_identities_test = []
print("Sampling between 0 and {} for train".format(numX*split_on_doc))
p = ratios_col[0:int(numX*split_on_doc)]
p = p / sum(p)
docID_train= [] #list of docID that was randomly chosen for each qid in train#
for i in range(datasize):
    cs_indices = np.random.choice(np.arange(0, numX*split_on_doc, dtype=int), size=cs_size, p=p) #doc index chosen for a qid#
    #print ("cs_indices", cs_indices)
    docID_train.append (cs_indices.tolist())
    cs_X = X[cs_indices] #features of chosen docs#
    cs_Y = Y[cs_indices] #rel of chosen docs#
    data_X.append(cs_X)
    data_Y.append(cs_Y)
    #print ("cs_X", cs_X)
    group_identities_train.append(cs_X[:,4]) #5th column (index=4) and all row. It is male column#
    #print ("group_identities_train", group_identities_train)

print("Sampling between {} and {} for test".format(numX*split_on_doc, numX))
p = ratios_col[int(numX*split_on_doc):]
p = p/sum(p)
docID_test= []
#for i in range(testsize): 
while len(docID_test) < testsize: #used while instead of for to make sure given "continue", we still have len(testsize) docs#
    cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
    group_id= X[cs_indices,4]
    if (len(set(group_id))==1 or 
        sum([Y[item] for i, item in enumerate(cs_indices) if group_id[i]==0]) == 0 or
        sum([Y[item] for i, item in enumerate(cs_indices) if group_id[i]==1]) == 0):
        continue
    docID_test.append (cs_indices.tolist())
    test_X.append(X[cs_indices])
    test_Y.append(Y[cs_indices])
    group_identities_test.append(X[cs_indices,4])



pickle.dump(docID_test, open( "german_docID_test_25.txt", "wb" ) )

pkl.dump((data_X, data_Y), open("german_train_rank_25.pkl", "wb"))
pkl.dump((test_X, test_Y), open("german_test_rank_25.pkl", "wb"))


In [ ]:
#Read data

docID_test = pickle.load( open( "german_docID_test_25.txt", "rb" ) )

with open('german_train_rank_25.pkl', 'rb') as f:
    data_train = pickle.load(f)


#same for data_test but with size 100 instead of 500
with open('german_test_rank_25.pkl', 'rb') as f:
     data_test = pickle.load(f)


In [ ]:
#Learn rel-scores

dr = YahooDataReader(None)
dr.data = pkl.load(open("german_train_rank_25.pkl", "rb"))
vdr = YahooDataReader(None)
vdr.data = pkl.load(open("german_test_rank_25.pkl","rb"))                  
model = linear_model.LinearRegression(fit_intercept=False, normalize=False)
feats, rel = dr.data
feats = np.array([item for sublist in feats for item in sublist])
rel = np.array([item for sublist in rel for item in sublist])
model.fit(feats, rel)
# predictions on validation
feats, rel = vdr.data
se_sum = 0
length = 0
predicted_rels = []
for i, query in enumerate(feats):
    rel_pred = model.predict(query)
    #print (rel_pred)
    predicted_rels.append(rel_pred)
    se_sum += np.sum((rel_pred - rel[i])**2)
    length += len(rel[i])

In [ ]:
#This code learn ranking based on fairness def of CTR, and evaluate based on fairness def of CTR with trust bias (MovieLense)
#Filter data to drop users and items with low number of interactions#
import random
import json
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from collections import defaultdict
from collections import Counter
import pickle as pkl
import pickle

def write_data(path, data):
    f = open(path, 'w')
    jsObj = json.dumps(data)
    f.write(jsObj)
    f.close()

def dataset_filtering(interaction, core):
    # filtering the dataset with core
    # movielens is filtered by only remaining only users with at least 20 interactions
    # we further filter the dataset by remaining users and items with at least 20 interactions
    user_id_dic = {}  # record the number of interaction for each user and item
    item_id_dic = {}
    for (user_id, item_id) in interaction:
        try:
            user_id_dic[user_id] += 1
        except:
            user_id_dic[user_id] = 1
        try:
            item_id_dic[item_id] += 1
        except:
            item_id_dic[item_id] = 1
    print ('#Original dataset')
    print('  User:', len(user_id_dic), 'Item:', len(item_id_dic))
    print('  User:', len(user_id_dic), 'Item:', len(item_id_dic), 'Interaction:', len(interaction), 'Sparsity:',
          100 - len(interaction) * 100.0 / len(user_id_dic) / len(item_id_dic), '%')
    sort_user = []
    sort_item = []
    for user_id in user_id_dic:
        sort_user.append((user_id, user_id_dic[user_id]))
    for item_id in item_id_dic:
        sort_item.append((item_id, item_id_dic[item_id]))
    sort_user.sort(key=lambda x: x[1])
    sort_item.sort(key=lambda x: x[1])
    print ('Fitering(core = ', core, '...', end = '')
    while sort_user[0][1] < core or sort_item[0][1] < core:
        # find out all users and items with less than core recorders
        user_LessThanCore = set()
        item_LessThanCore = set()
        for pair in sort_user:
            if pair[1] < core:
                user_LessThanCore.add(pair[0])
            else:
                break
        for pair in sort_item:
            if pair[1] < core:
                item_LessThanCore.add(pair[0])
            else:
                break
        # reconstruct the interaction record, remove the cool one
        interaction_filtered = []
        for (user_id, item_id) in interaction:
            if not (user_id in user_LessThanCore or item_id in item_LessThanCore):
                interaction_filtered.append((user_id, item_id))
        # update the record
        interaction = interaction_filtered

        # count the number of each user and item in new data, check if all cool users and items are removed
        # reset all memory variables
        user_id_dic = {}  # record the number of interaction for each user and item
        item_id_dic = {}
        for (user_id, item_id) in interaction:
            try:
                user_id_dic[user_id] += 1
            except:
                user_id_dic[user_id] = 1
            try:
                item_id_dic[item_id] += 1
            except:
                item_id_dic[item_id] = 1

        sort_user = []
        sort_item = []
        for user_id in user_id_dic:
            sort_user.append((user_id, user_id_dic[user_id]))
        for item_id in item_id_dic:
            sort_item.append((item_id, item_id_dic[item_id]))
        sort_user.sort(key=lambda x: x[1])
        sort_item.sort(key=lambda x: x[1])
        print (len(interaction), end = ' ')
    print()
    print ('#Filtered dataset')
    print ('  User:', len(user_id_dic), 'Item:', len(item_id_dic), 'Interaction:', len(interaction), 'Sparsity:', 100 - len(interaction) * 100.0 / len(user_id_dic) / len(item_id_dic), '%')
    return interaction



#read interaction data
core = 80
cold_thre = 15



path_read_inter = '../../../MovieLense/ml-100k/u.data'
f_inter = open(path_read_inter, "r")
data_inter = f_inter.read()
f_inter.close()
Interaction = []

data_inter = data_inter.split('\n')
#print (data_inter)
for interactions in data_inter:
    #interactions = interactions.split('::')
    interactions = interactions.split('\t')
    #print ("interaction", interactions)
    if len(interactions) > 1:
        Interaction.append((interactions[0], interactions[1]))
#Interaction = list(set(Interaction))

#print("1", len(Interaction))
Interaction = dataset_filtering(Interaction, core)
#print ("2", len(Interaction))


remained_items=[]
for el in range(len(Interaction)):
    remained_items.append(Interaction[el][1])
#print (set(remained_items))

In [ ]:
#Find less popular movies

dataset = pd.read_csv('../../../MovieLense/ml-100k/u.data', sep='\t')
dataset.head()

movieDic={}
movie_list=[]


for i in range(len(Interaction)):    ##if want to decide the least popular after pruning cold users/items
    movie_list.append(Interaction[i][1])
#print (len(set(movie_list)))

res = Counter(movie_list)
# print (res)

least_rated= res.most_common()[-70:]
least_rated_movies=[]
for i in range(len(least_rated)):
    least_rated_movies.append((str(least_rated[i][0])))
# print (least_rated_movies)
FemaleID_str= least_rated_movies

In [ ]:
#Write data

#processs interaction data
user_list= []
item_list= []
for i in range(len(Interaction)):
    user_list.append(int(Interaction[i][0]))
    item_list.append(int(Interaction[i][1]))
user_set= sorted(set(user_list))
item_set= sorted(set(item_list))
#print ("user_set",user_set)
#print ("item_set",item_set)

D={} #dictionaey with key=user and value=items that user interacted with
D = defaultdict(list)
for j in range(len(Interaction)):
    D[int(Interaction[j][0])].append(int(Interaction[j][1]))
#print (len(D))

#process user data and item data (dropped zipcode as it cannot be either scaler or onehotencoding)
data_user= pd.read_csv('../../../MovieLense/ml-100k/u.user', sep='|', 
                  names=["user_id", "age", "gender", "job","zipcode"])
data_user= data_user.drop(columns=['zipcode'])
print ("data_user", data_user)

data_item= pd.read_csv('../../../MovieLense/ml-100k/u.item', sep='|', names=["movie_id", "movie_title", "release_date", "video release_date", "IMDb URL", "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"], encoding='latin-1')
#print ("data_item",data_item)
data_item= data_item.drop(columns=['movie_title', 'video release_date','IMDb URL'])
data_item['release_date'] = data_item['release_date'].str[-4:]
#print ("data_item",data_item)


#add popularity column to data_item panda

data_item['unpopular']= data_item['movie_id'].map(lambda x: 1 if str(x) in FemaleID_str else 0)
print(data_item)

In [ ]:
#Concatenate user-item features

DATA=[]
feature_list=[]
label_list=[]
for user in sorted(random.sample(D.keys(), 150)):  #choose 100 users randomly
    label_array=[]
    for item in sorted(random.sample(item_set, 150)): #choose 100 items randomly
        df_user= data_user.loc[data_user['user_id'] == user]
        df_user= df_user.reset_index(drop=True)
        df_item= data_item.loc[data_item['movie_id'] == item]
        df_item= df_item.reset_index(drop=True)
        feature_list.append(pd.concat([df_user,df_item], axis=1))
        if item in D[user]:
            label_array.append(1)  
        else:
            label_array.append(0)
    label_list.append(np.array(label_array))
    

In [ ]:
feature_list = pd.concat(feature_list) #change the feature list that contains multiple dataframes, to a single dataframe with single head and multiplee rows(vertical) 
feature_list= feature_list.drop(columns=['user_id', 'movie_id'])
preprocess = make_column_transformer((StandardScaler(),['age','release_date']), 
                                     (OneHotEncoder(sparse=False), ['gender', 'job', 'unpopular']), remainder='passthrough')
feature_list = preprocess.fit_transform(feature_list)
feature_list= np.split(feature_list, 150)

In [ ]:
#split data based on interaction (not user)

data_X = []
data_Y = []
test_X = []
test_Y = []
docID_test = []

for u in range(len(label_list)):
    all_index = list(range(len(label_list[u])))
    train_index = set(random.sample(all_index, int(0.8* len(all_index))))
    test_index = list(set(all_index) - train_index)
    train_index = list(train_index)
    data_X.append(feature_list[u][train_index])
    data_Y.append(label_list[u][train_index])
    test_X.append(feature_list[u][test_index])
    test_Y.append(label_list[u][test_index])
    docID_test.append(np.take(item_set, test_index).tolist())






#remove qid with sum_female=0 or sum_male=0
F_rel=0
M_rel=0
unwanted_u=[]

for u in range(len(docID_test)):
    F_rel=0
    M_rel=0
    for doc in range(len(docID_test[0])):
        if str(docID_test[u][doc]) in least_rated_movies:
            F_rel= F_rel+ test_Y[u][doc]  
        else:
            M_rel= M_rel+ test_Y[u][doc]
    if F_rel==0 or M_rel==0:
        unwanted_u.append(u)


test_X_ = []
test_Y_ = []
docID_test_=[]
for i in range(len(test_X)):
    if i not in unwanted_u:
        test_X_.append(test_X[i])
        test_Y_.append(test_Y[i])
        docID_test_.append(docID_test[i])
test_X= test_X_
test_Y= test_Y_
docID_test= docID_test_



pickle.dump(docID_test, open( "MovieLense_docID_test_25.txt", "wb" ) )

pkl.dump((data_X, data_Y), open("MovieLense_train_rank_25.pkl", "wb"))
pkl.dump((test_X, test_Y), open("MovieLense_test_rank_25.pkl", "wb"))

In [ ]:
#Read data

docID_test = pickle.load( open( "MovieLense_docID_test_25.txt", "rb" ) )

with open('MovieLense_train_rank_25.pkl', 'rb') as f:
    data_train = pkl.load(f)

with open('MovieLense_test_rank_25.pkl', 'rb') as f:
     data_test = pickle.load(f)
        

In [ ]:
#Learn Rel-scores

from YahooDataReader import YahooDataReader
from sklearn.linear_model import LinearRegression
from sklearn import linear_model


dr = YahooDataReader(None)
dr.data = pkl.load(open("MovieLense_train_rank_25.pkl", "rb"))
vdr = YahooDataReader(None)
vdr.data = pkl.load(open("MovieLense_test_rank_25.pkl","rb"))                  
model = linear_model.LinearRegression(fit_intercept=False, normalize=False)
feats, rel = dr.data
feats = np.array([item for sublist in feats for item in sublist])
rel = np.array([item for sublist in rel for item in sublist])
model.fit(feats, rel)
feats, rel = vdr.data
se_sum = 0
length = 0
predicted_rels = []
for i, query in enumerate(feats):
    rel_pred = model.predict(query)
    predicted_rels.append(rel_pred)
    se_sum += np.sum((rel_pred - rel[i])**2)
    length += len(rel[i])


In [ ]:
#Main Code


#We have nothing to do with train anymore. Now we have test data consisting of 100 qid each with 25 documents.#
#Given the docIDs for each qid, in docID_test , predicted_relScores for each docID, in predicted_rels, plus group identities, FORGE provide fair ranking



docID_test_str=[[] for _ in range(len(docID_test))] #type=str
size_female_test = []
size_male_test=[]
sum_rel_female=[]
sum_rel_male= []
sum_rel_totall=[]
for qid in range(len(docID_test)):
    #print ("qid", qid)
    F_size= 0
    M_size= 0
    F_rel=0
    M_rel=0
    for doc in range(len(docID_test[0])):
        docID_test_str[qid].append(str(docID_test[qid][doc]))
        if str(docID_test[qid][doc]) in FemaleID_str:
            F_size=F_size+1
            F_rel= F_rel+ data_test[1][qid][doc]  # predicted_rels[qid][doc] if assume precietd rel is ground truth and data_test[1][qid][doc] O.W
        else:
            M_size= M_size+1
            M_rel= M_rel+ data_test[1][qid][doc]  # predicted_rels[qid][doc] if assume precietd rel is ground truth and data_test[1][qid][doc] O.W 
    
    size_female_test.append(F_size) #number of female docs at each query
    size_male_test.append(M_size)
    totall_size= len (docID_test_str[qid])
    sum_rel_female.append(F_rel) #sum of rel score of female at each query
    sum_rel_male.append(M_rel)
    sum_rel_totall.append(sum_rel_female[qid]+ sum_rel_male[qid])


In [ ]:
def KL_Divergence(p1,p2,q1,q2): #can use jenson divergance that is bounded between 0 and 1#
    if p1!=0 and p2!=0:
        J=(    (p1*np.log((2*p1)/(p1+q1)))  +  (p2*np.log((2*p2)/(p2+q2)))    +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    elif p1==0:
        J=(    (p2*np.log((2*p2)/(p2+q2)))    +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    elif p2==0:

        J=(   (p1*np.log((2*p1)/(p1+q1)))  +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    else:
        J=(   (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))   )/ 2 
    return J

In [ ]:
def fairness(size_docID, 
             size_female, 
             size_male, 
             current_male_CTR, 
             sofar_male_CTR, 
             current_female_CTR, 
             sofar_female_CTR,
             ):


    CTR_male = sofar_male_CTR + current_male_CTR
    CTR_female =  sofar_female_CTR + current_female_CTR
    CTR_total = CTR_male + CTR_female
    current_fairness= 1- KL_Divergence(CTR_male / CTR_total,
                                       CTR_female / CTR_total,
                                       size_male/size_docID, 
                                       size_female/size_docID
                                       )
    return current_fairness

In [ ]:
def separate_and_sort(
    docID,
    rel_scores
    ):
    """
    Arg list:
    movieID: list of movie IDs. Here, list of all docID for a certain qid
    rel_scores: dict() that maps movie_ID -> rel_score
    """
    female_sorted_by_rel = []
    male_sorted_by_rel = []
    
    for i in docID:    
        if i in FemaleID_str:
            female_sorted_by_rel.append(i)
        else:
            male_sorted_by_rel.append(i)

    female_sorted_by_rel.sort(key = lambda x : rel_scores[x], reverse=True) 
#     print ("female_sorted_by_rel", female_sorted_by_rel)
    male_sorted_by_rel.sort(key = lambda x : rel_scores[x], reverse=True)
    #print ("male_sorted_by_rel",male_sorted_by_rel)

    return male_sorted_by_rel, female_sorted_by_rel

In [ ]:

def interpolation_optimized(
    qid,
    docID,
    rel_scores,
    Z,
):
    """
    Arg list:
    docID: list of docIDs for each qid in test data
    rel_scores: dict() that maps doc_ID -> rel_score
    Z: Z value for interpolation
    """
    all_sorted_by_rel= sorted(docID, key = lambda x : rel_scores[x], reverse=True)
#     print ("all_sorted_by_rel", len(all_sorted_by_rel))
#     print('rel_scores', rel_scores)
    S=[]
    sofar_female_CTR=0
    sofar_male_CTR=0
    sofar_DCG=0
    IDCG=0
    
    availablity = all_sorted_by_rel[:] #make a copy of all_sorted to avoid del make problem for IDCG
    while len (S)< 25:
        #print ("S", len(S))
        IDCG= IDCG+ (float(2**float(rel_scores[all_sorted_by_rel[len(S)]])-1) /  math.log2(1+len(S)+1))
        epsilon_1= 0.65
        epsilon_plus= 1- ((len(S)+2)/100)
        epsilon_minus= epsilon_1* (1/min( len(S)+1, 10) )
        max_intpol_score = 0
        max_item_data = None
        for item in availablity:
            DCG = sofar_DCG+ (float(2**float(rel_scores[item])-1) /  math.log2(1+len(S)+1))
            nDCG =  float (DCG)/ IDCG
            if item in FemaleID_str:
                #print ("FemaleID_str is available")
                current_male_CTR = 0
                current_female_CTR = float(  (rel_scores[item]* epsilon_plus) + ( (1-rel_scores[item]) * epsilon_minus)  ) * 1/math.log2(1+len(S)+1)
            else:
                current_male_CTR =   float(  (rel_scores[item]* epsilon_plus) + ( (1-rel_scores[item]) * epsilon_minus)  ) * 1/math.log2(1+len(S)+1)
                current_female_CTR = 0
            #print ("fairnes_component", sum_rel_totall[qid], sum_rel_female[qid], sum_rel_male[qid], current_male_CTR, sofar_male_CTR, current_female_CTR, sofar_female_CTR)
            fair_metric = fairness(sum_rel_totall[qid], sum_rel_female[qid], sum_rel_male[qid], current_male_CTR, sofar_male_CTR, current_female_CTR, sofar_female_CTR)
            #print ("fair_metric", fair_metric)
            intpol_score = (1-Z) * nDCG + Z * fair_metric
            if intpol_score > max_intpol_score:
                max_intpol_score = intpol_score
                max_item_data = (item, current_female_CTR, current_male_CTR, DCG)


        #print("max_item_data", max_item_data)
        S.append(max_item_data[0])
        availablity.remove(max_item_data[0])
        sofar_female_CTR += max_item_data[1]
        sofar_male_CTR += max_item_data[2]
        sofar_DCG = max_item_data[3]
        
    #print ("S", S)
    return S

In [ ]:

def get_nDCG(docID, rel_scores, item_list):
    """
    gives the nDCG of ranking
    input:
    docID: id of documents
    rel_scores: relevant scores assuming predict = ground truth or not
    item_list: dictionary key= docID, value= relscores
    """
    sorted_docID= sorted(docID, key=lambda x:rel_scores [docID.index(x)] , reverse=True)   #a: b means a is input and will be sorted based on b. Here x= docID that will be sorted based on rel scores#
    Denom= float(0)
    Nom= float(0)
    for i in range (len(item_list)):
        temp1= D_real[str(sorted_docID[i])] #D1 if want to assume predicted rel is ground truth, D_real otherwise#
        temp2= 2**(float (temp1))
        Denom= Denom+(   (temp2-1)  / (math.log2(i+2))    )
        temp3= D_real[str(item_list[i])]    #D1 if want to assume predicted rel is ground truth, D_real otherwise#
        temp4= 2**(float (temp3))
        Nom=Nom + (  (temp4-1)  / (math.log2(i+2))    )
    nDCG= (float(Nom)/float(Denom))
    return (nDCG, sorted_docID  )

In [ ]:
def get_divergence (qid, docID, rel_scores):
    sofar_female_CTR=0
    sofar_male_CTR=0
    D_movie_list=[]
    for i in range(len(docID)):
        epsilon_1= 0.65
        epsilon_plus= 1- ((i+2)/100)
        epsilon_minus= epsilon_1* ( 1/min (i+1, 10) )
        if docID[i] in FemaleID_str:
            sofar_female_CTR += float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        else:
            sofar_male_CTR +=   float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        
        CTR_total= sofar_female_CTR+ sofar_male_CTR  
        D_movie_list.append(KL_Divergence(sofar_male_CTR/ CTR_total, sofar_female_CTR/ CTR_total , sum_rel_male[qid]/sum_rel_totall[qid], sum_rel_female[qid]/sum_rel_totall[qid]))
    return (D_movie_list)

In [ ]:

import math

interplotion_value=[1]
for z in interplotion_value:
    D2={} # a dictionary: key= user id, value= S/item_list

    nDCG_all_q=[]
    D_item_list_all_q=[]

    #print ("len-docid-test", len(docID_test))
    for qid in range(len(docID_test)):
        nDCG=[]
        nDiv=[]
        #print ("qid", qid)
        D1={} # a dictionary: key= movieID, value= predicted rel_scores
        D_real= {} # a dictionary: key= movieID, value= true rel_scores
        for i in range (len(docID_test[0])):
            D1[docID_test_str[qid][i]]= predicted_rels[qid][i]
            D_real[docID_test_str[qid][i]]= data_test[1][qid][i]
        item_list  = interpolation_optimized (qid,docID_test_str[qid], D1, z)
        for i in range(len(item_list)):
            nDCG.append(get_nDCG(docID_test[qid], data_test[1][qid], item_list[0:i+1])[0]) #predicted_rels[qid] if assume precietd rel is ground truth, data_test[1][qid] Otherwise
            D_item_list= get_divergence(qid, item_list[0:i+1], D_real)  #D1 if we assume predicted=true rel, and D_real o.w


        nDCG_all_q.append (nDCG)
        D_item_list_all_q.append(D_item_list)



    avg_nDCG_at_indx_all_q= np.mean(nDCG_all_q, axis=0)
    avg_D_item_list_at_indx_all_q= np.mean(D_item_list_all_q, axis=0)


    print ("avg_nDCG_at_indx_all_q", avg_nDCG_at_indx_all_q[0])
    print ("avg_D_item_list_at_indx_all_q", avg_D_item_list_at_indx_all_q[0])

